In [2]:
import requests
import pandas
from lxml import etree
import execjs
import re
import csv

In [3]:
# 基本信息 时间 里程 排量 现价 原价 年份

In [120]:
headers = {
    'Connection': 'keep-alive',
    'Pragma': 'no-cache',
    'Cache-Control': 'no-cache',
    'Upgrade-Insecure-Requests': '1',
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.88 Safari/537.36',
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',
    'Cookie': 'antipas=63020N2bm6T637783h82u0g; uuid=3cdc7747-40f6-4d00-a30a-084d0071e915; clueSourceCode=%2A%2300; ganji_uuid=1406136594505570875994; sessionid=3897d615-441d-4a85-bbe6-01b484364817; lg=1; cainfo=%7B%22ca_a%22%3A%22-%22%2C%22ca_b%22%3A%22-%22%2C%22ca_s%22%3A%22self%22%2C%22ca_n%22%3A%22self%22%2C%22ca_medium%22%3A%22-%22%2C%22ca_term%22%3A%22-%22%2C%22ca_content%22%3A%22-%22%2C%22ca_campaign%22%3A%22-%22%2C%22ca_kw%22%3A%22-%22%2C%22ca_i%22%3A%22-%22%2C%22scode%22%3A%22-%22%2C%22keyword%22%3A%22-%22%2C%22ca_keywordid%22%3A%22-%22%2C%22display_finance_flag%22%3A%22-%22%2C%22platform%22%3A%221%22%2C%22version%22%3A1%2C%22client_ab%22%3A%22-%22%2C%22guid%22%3A%223cdc7747-40f6-4d00-a30a-084d0071e915%22%2C%22ca_city%22%3A%22cq%22%2C%22sessionid%22%3A%223897d615-441d-4a85-bbe6-01b484364817%22%7D; cityDomain=hz; user_city_id=26; preTime=%7B%22last%22%3A1609606289%2C%22this%22%3A1609605332%2C%22pre%22%3A1609605332%7D',
}

In [83]:
value_search = re.compile(r"anti\('(.*?)','(.*?)'\);")

In [84]:
with open('js.js', 'r') as rf:
    js_code=rf.read()
js = execjs.compile(js_code)

In [125]:
with open('data.csv', 'w', encoding='utf-8-sig', newline='') as wf:
    writer = csv.writer(wf)
    writer.writerow(['基本信息', '时间', '里程', '现价', '原价', '排量', '年份'])

In [108]:
def my_requests(url, headers):
    r = requests.get(url, headers=headers)
    while r.status_code == 203:
        try:
            string = value_search.search(r.text).group(1)
            key = value_search.search(r.text).group(2)
        except:
            print(r.status_code)
            return 'no'
        js_return = js.call("anti", string, key)  # 传递的是当前需要使用的"anti"方法，和两个方法中需要的参数
        # print(js_return)  # 输出anti方法进行加密处理后的cookie值
        print(js_return)
        cookie_value = "antipas=" + js_return  # 该cookie格式需要在前面加"antipas="字符串
        headers["Cookie"] = cookie_value  # 将cookie设置到头部
        r = requests.get(url, headers=headers)
    return r

In [126]:
def crawl_detail(url, infos_list):
    r = my_requests(url, headers)
    if r == 'no':
        return
    html = etree.HTML(r.text)
    pailiang = html.xpath('/html/body/div[4]/div[3]/div[2]/ul/li[3]/span/text()')[0]
    infos_list.append(pailiang)
    infos_list.append('待处理')
    print(infos_list)
    with open('data.csv', 'a', encoding='utf-8-sig', newline='') as af:
        write = csv.writer(af)
        write.writerow(infos_list)

In [127]:
url = 'https://www.guazi.com/hz/buy/o{}'

In [128]:
for i in range(1, 51):
    r = my_requests(url.format(i), headers=headers)
    with open('r.html', 'w', encoding='utf-8') as wf:
        wf.write(r.text)
    print(r.url)
    if r == 'no':
        continue
    html = etree.HTML(r.text)
    lis = html.xpath('/html/body/div[6]/ul/li')
    for li in lis:
        new_url = li.xpath('./a/@href')[0]
        print(new_url)
        base_info = li.xpath('./a/h2/text()')[0]
        tmp = li.xpath('./a/div[1]/text()')
        year, licheng = tmp[0], tmp[1]
        now_price = ''.join(li.xpath('./a/div[2]/p//text()')).strip()
        try:
            old_price = li.xpath('./a/div[2]/em/text()')[0]
        except:
            old_price = '未知'
        infos_list = [base_info, year, licheng, now_price, old_price]
        crawl_detail('https://www.guazi.com' + new_url, infos_list)
    print('当前页面: {}'.format(i))

https://www.guazi.com/hz/buy/o1
/hz/2f766701cdf6e6f2x.htm#fr_page=list&fr_pos=city&fr_no=0
['大众 途观 2012款 1.8TSI 自动两驱菁英版', '2012年', '8.8万公里', '9.20万', '27.66万', '1.8T', '待处理']
/hz/d2af556a58870333x.htm#fr_page=list&fr_pos=city&fr_no=1
['大众 朗逸 2013款 改款 1.4TSI DSG舒适版', '2013年', '8.0万公里', '5.40万', '16.05万', '1.4T', '待处理']
/hz/b367796b41070d17x.htm#fr_page=list&fr_pos=city&fr_no=2
['日产 骊威 2015款 1.6XE CVT舒适版', '2017年', '4.5万公里', '6.20万', '10.40万', '1.6L', '待处理']
/hz/e1041829e3312bf1x.htm#fr_page=list&fr_pos=city&fr_no=3
['现代ix35 2010款 2.0L 自动两驱精英版GLS', '2012年', '12.5万公里', '6.18万', '21.25万', '2.0L', '待处理']
/hz/a92d205706591ad3x.htm#fr_page=list&fr_pos=city&fr_no=4
['斯柯达 明锐 2009款 2.0L 手动逸仕版', '2009年', '9.3万公里', '2.03万', '14.97万', '2.0L', '待处理']
/hz/2ff7766df137d3cex.htm#fr_page=list&fr_pos=city&fr_no=5
['日产 逍客 2010款 20X雷 CVT 2WD', '2010年', '10.7万公里', '5.20万', '20.49万', '2.0L', '待处理']
/hz/05315310ee06983fx.htm#fr_page=list&fr_pos=city&fr_no=6
['奔驰S级 2010款 S 300 L 商务型(进口)', '2011年', '14.7万公里', '

['别克 君威 2015款 1.6T 领先技术型', '2016年', '6.2万公里', '8.78万', '20.61万', '1.6T', '待处理']
/hz/e0e95cb7a616ee22x.htm#fr_page=list&fr_pos=city&fr_no=18
['现代ix25 2017款 1.6L 自动智能型', '2017年', '1.4万公里', '8.68万', '14.42万', '1.6L', '待处理']
/hz/3fa44189e75be84cx.htm#fr_page=list&fr_pos=city&fr_no=19
['斯柯达 晶锐 2012款 1.4L 自动晶致版', '2011年', '11.1万公里', '3.38万', '9.87万', '1.4L', '待处理']
/hz/f95566624841df16x.htm#fr_page=list&fr_pos=city&fr_no=20
['Jeep 指南者 2017款 200T 自动家享版', '2017年', '5.5万公里', '10.20万', '19.52万', '1.4T', '待处理']
/hz/4b27f9ae0ff80d39x.htm#fr_page=list&fr_pos=city&fr_no=21
['现代 索纳塔九 2016款 2.0L hybrid HS领先型', '2017年', '3.3万公里', '10.98万', '24.94万', '2.0L', '待处理']
/hz/bb924b72e8bbf70cx.htm#fr_page=list&fr_pos=city&fr_no=22
['路虎 神行者2 2011款 3.2L i6 HSE汽油版(进口)', '2011年', '14.8万公里', '12.80万', '71.42万', '3.2L', '待处理']
/hz/f124de28f7be513ax.htm#fr_page=list&fr_pos=city&fr_no=23
['奔驰GLK级 2013款 改款 GLK 300 4MATIC 时尚型', '2014年', '14.3万公里', '19.98万', '52.65万', '3.0L', '待处理']
/hz/ce85a70ef8a0df5bx.htm#fr_page=list

['启辰T90 2018款 2.0L CVT智联智尚版 国V', '2019年', '1.9万公里', '8.50万', '14.96万', '2.0L', '待处理']
/hz/723cdf5e0772ba0ex.htm#fr_page=list&fr_pos=city&fr_no=35
['路虎 揽胜星脉 2017款 P250 S(进口)', '2018年', '5.9万公里', '42.80万', '74.68万', '2.0T', '待处理']
/hz/af9b0f930ec51066x.htm#fr_page=list&fr_pos=city&fr_no=36
['奔驰C级 2017款 改款 C 180 L 时尚型运动版', '2017年', '6.1万公里', '24.80万', '35.36万', '1.6T', '待处理']
/hz/63583e4484306c61x.htm#fr_page=list&fr_pos=city&fr_no=37
['奔驰GLC级 2017款 GLC 260 4MATIC 豪华型', '2017年', '1.0万公里', '34.30万', '48.30万', '2.0T', '待处理']
/hz/465ccbb4a7828c40x.htm#fr_page=list&fr_pos=city&fr_no=38
['奔驰E级 2019款 E 260 L 运动型', '2019年', '1.2万公里', '39.80万', '46.22万', '1.5T', '待处理']
/hz/f3a277df854fc1fcx.htm#fr_page=list&fr_pos=city&fr_no=39
['奥迪A3 2014款 Limousine 35 TFSI 自动时尚型', '2014年', '6.9万公里', '10.99万', '24.41万', '1.4T', '待处理']
当前页面: 3
https://www.guazi.com/hz/buy/o4
/qd/c852b50d280caf12x.htm#fr_page=list&fr_pos=city&fr_no=0
['别克 凯越 2013款 1.5L 自动尊享型', '2015年', '2.5万公里', '4.57万', '12.58万', '1.5L', '待处理']
/

['福特 翼搏 2013款 1.0L GTDi 手动尊贵型', '2015年', '7.0万公里', '4.45万', '13.00万', '1.0T', '待处理']
/wenzhou/5cadac00e5d3c10ax.htm#fr_page=list&fr_pos=city&fr_no=12
['奥迪Q7 2013款 35 TFSI 进取型(进口)', '2014年', '9.1万公里', '31.98万', '89.12万', '3.0T', '待处理']
/nj/15e659874e6838c8x.htm#fr_page=list&fr_pos=city&fr_no=13
['大众 朗逸 2013款 改款经典 1.6L 手动舒适版', '2016年', '5.4万公里', '6.10万', '13.67万', '1.6L', '待处理']
/sjz/dff06c06f10f2feex.htm#fr_page=list&fr_pos=city&fr_no=14
['凯迪拉克SRX 2013款 3.0L 精英型(进口)', '2013年', '15.2万公里', '12.70万', '51.00万', '3.0L', '待处理']
/xm/b0c37b8b0c7f5c54x.htm#fr_page=list&fr_pos=city&fr_no=15
['雪佛兰 赛欧 2011款 三厢 1.4L 手动幸福版', '2012年', '5.8万公里', '1.88万', '7.85万', '1.4L', '待处理']
/nj/5b9db3575aaeb95ex.htm#fr_page=list&fr_pos=city&fr_no=16
['福特 福克斯 2012款 两厢 1.6L 手动舒适型', '2014年', '6.1万公里', '4.48万', '13.01万', '1.6L', '待处理']
/dl/17de158d98d45bd9x.htm#fr_page=list&fr_pos=city&fr_no=17
['日产 奇骏 2014款 2.5L CVT豪华版 4WD', '2015年', '8.6万公里', '13.98万', '25.92万', '2.5L', '待处理']
/jilin/8a9a52fa410fe203x.htm#fr_page=lis

['凯迪拉克ATS-L 2017款 28T 技术型', '2019年', '1.7万公里', '19.10万', '29.72万', '2.0T', '待处理']
/cc/c9322cdeb48b08efx.htm#fr_page=list&fr_pos=city&fr_no=29
['MINI Clubman 2018款 1.5T COOPER 赛车手', '2018年', '2.2万公里', '24.50万', '30.81万', '1.5T', '待处理']
/su/abf16ac48debd64fx.htm#fr_page=list&fr_pos=city&fr_no=30
['现代 途胜 2015款 1.6T 双离合两驱领先型', '2016年', '6.9万公里', '9.90万', '21.37万', '1.6T', '待处理']
/cd/0ed8f76d268fc3ebx.htm#fr_page=list&fr_pos=city&fr_no=31
['日产 骊威 2007款 1.6L 自动超能型', '2009年', '13.8万公里', '2.75万', '11.27万', '1.6L', '待处理']
/jn/aa35b6bc8702dd14x.htm#fr_page=list&fr_pos=city&fr_no=32
['奥迪A4L 2012款 1.8 TFSI 手动舒适型', '2011年', '3.5万公里', '8.88万', '29.61万', '1.8T', '待处理']
/dl/fee08977e8b82766x.htm#fr_page=list&fr_pos=city&fr_no=33
['丰田 雷凌 2016款 1.6G CVT精英版', '2016年', '6.6万公里', '8.30万', '13.55万', '1.6L', '待处理']
/yc/45e238c2962677fex.htm#fr_page=list&fr_pos=city&fr_no=34
63K02m0m2g663778e3i82uf0I
63B02z02Mp6i63778382GA0
63Y02P02f6u637783p82hhX0
6302u02h6qQ63778v382e0w
630202aC6UB637783E82Lzw0K
63a0202q6JY

['福特 福睿斯 2015款 1.5L 自动时尚型', '2017年', '4.9万公里', '6.35万', '13.00万', '1.5L', '待处理']
/cs/d654b1b66f1695c5x.htm#fr_page=list&fr_pos=city&fr_no=2
['起亚K2 2012款 三厢 1.6L AT Premium纪念版', '2013年', '7.3万公里', '4.78万', '10.84万', '1.6L', '待处理']
/zhuhai/bcd68f4b39bd428cx.htm#fr_page=list&fr_pos=city&fr_no=3
['本田 思域 2016款 220TURBO CVT豪华版', '2017年', '4.0万公里', '12.68万', '15.19万', '1.5T', '待处理']
/km/8c3881ae37e89476x.htm#fr_page=list&fr_pos=city&fr_no=4
['雪佛兰 迈锐宝XL 2016款 1.5T 双离合锐尚版', '2016年', '6.4万公里', '9.16万', '22.78万', '1.5T', '待处理']
/fz/36db0b9ae05867bdx.htm#fr_page=list&fr_pos=city&fr_no=5
['长安CS35 2012款 1.6L 自动豪华型 国V', '2013年', '7.3万公里', '4.00万', '10.19万', '1.6L', '待处理']
/su/5b0580b527cf6499x.htm#fr_page=list&fr_pos=city&fr_no=6
['雪佛兰 迈锐宝 2014款 1.6T 自动舒适版', '2015年', '12.7万公里', '8.28万', '19.74万', '1.6T', '待处理']
/xa/3152de8d75d82bbdx.htm#fr_page=list&fr_pos=city&fr_no=7
['奔驰GLA级 2016款 GLA 200 时尚型', '2017年', '4.3万公里', '18.98万', '31.46万', '1.6T', '待处理']
/foshan/29da24241cf499f3x.htm#fr_page=list&fr_pos=

KeyboardInterrupt: 